In [ ]:
# importing all libraries 
import numpy as np    
import pandas as pd
import os 
import cv2 as cv
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Rescaling,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# converting data into train data from image generator
train_data = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/state-farm-distracted-driver-detection/imgs/train',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(100, 100),
  batch_size=128,label_mode='categorical',)


In [ ]:
# converting data into validation data from image generator
val_data = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/state-farm-distracted-driver-detection/imgs/train',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(100, 100),
  batch_size=128,label_mode='categorical',)

In [ ]:
# showing some of the images
classes = train_data.class_names
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for images,labels in train_data.take(1):
    labels = labels.numpy()
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(classes[labels[i].argmax()])
        plt.axis("off")

In [ ]:
# creating our model
model = tf.keras.models.Sequential([
    Rescaling(scale = 1/255,input_shape=(100,100,3)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.1),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.1),
    Flatten(),
    Dense(1024,activation='relu'),
    Dropout(0.1),
    Dense(512,activation='relu'),
    
    Dense(256,activation='relu'),
    Dropout(0.1),
    Dense(10,activation='softmax'),
])

# compiling our model
model.compile(optimizer = Adam(lr=0.01),loss = 'categorical_crossentropy',metrics=['acc'])
model.summary()

In [ ]:
# early stopping to stop overfitting
es = EarlyStopping(monitor='val_acc',min_delta=0.01,patience=2)


In [ ]:
# fitting the model
history = model.fit(train_data,epochs=10,validation_data=val_data,callbacks=[es])

In [ ]:
# visualizing accuracy and losses
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = history.epoch

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# converting training data from image generator for prediction
test_data = image_dataset_from_directory(
    '/kaggle/input/state-farm-distracted-driver-detection/imgs/test',
    batch_size = 128,
    image_size=(100,100),
    labels = None,
    label_mode=None,
    shuffle = False
)

In [ ]:
# function for predicting images
def predict_image(path):
    img = tf.keras.utils.load_img(path).resize((100,100))
    img = np.array(img).reshape((1,100,100,3))
    y = model.predict(img,verbose=False)
    return y
    

In [ ]:
# predicting some test images
test_path = '/kaggle/input/state-farm-distracted-driver-detection/imgs/test'
plt.figure(figsize=(10,10))
i=1
for img_path in os.listdir(test_path)[:25]:
    img_path = os.path.join(test_path,img_path)
    img = tf.keras.utils.load_img(img_path)
    ax = plt.subplot(5, 5, i)
    plt.imshow(img)
    plt.title('c'+str(predict_image(img_path).argmax()))
    plt.axis("off")
    i += 1

In [ ]:
# also use for predicting test data takes more time
# y = np.zeros((79726,10))
# test_dir = '/kaggle/input/state-farm-distracted-driver-detection/imgs/test'
# count = 0
# for i in tqdm(os.listdir(test_dir)):
#     path = os.path.join(test_dir,i)
#     y[count] = predict_image(path)
#     count += 1

In [ ]:
#predicting test data given
y = model.predict(test_data)

In [ ]:
y.shape

In [ ]:
# exporting data as given
df = pd.DataFrame(y)
df.columns = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
filepath = [i.split('/')[-1] for i in test_data.file_paths]
df1 = pd.DataFrame(filepath)
df1.columns = ['img']
df = df1.join(df)
df.to_csv('/kaggle/working/output.csv',index=False)

In [ ]:
df.head()